In [1]:
##################################################
## Binance Python Bot :]
##################################################

import time
# pip install python-binance
from binance.client import Client
from binance.enums import *
# pip install pandas
import pandas as pd
# pip install numpy
import numpy as np
import datetime as dt

In [2]:
# Functions
def computeRSI (data, time_window):
    diff = np.diff(data)
    up_chg = 0 * diff
    down_chg = 0 * diff
    
    # up change is equal to the positive difference, otherwise equal to zero
    up_chg[diff > 0] = diff[ diff>0 ]
    
    # down change is equal to negative deifference, otherwise equal to zero
    down_chg[diff < 0] = diff[ diff < 0 ]

    up_chg = pd.DataFrame(up_chg)
    down_chg = pd.DataFrame(down_chg)
    
    up_chg_avg   = up_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    down_chg_avg = down_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    
    rs = abs(up_chg_avg/down_chg_avg)
    rsi = 100 - 100/(1+rs)
    rsi = int(rsi[0].iloc[-1])
    return rsi

In [3]:
def bollingerband(symbol,width,intervalunit,length):
    if intervalunit=='1T':
        start_str='100 minutes ago UTC'
        interval_data='1m'
        
        D=pd.DataFrame(client.get_historical_klines(symbol=symbol,start_str=start_str,interval=interval_data))
        D.columns=['open_time','open','high','low','close','volume','close_time','qav','num_trades','taker_base_vol','taker_quote_vol','is_best_match']
        D['open_date_time']=[dt.datetime.fromtimestamp(x/1000) for x in D.open_time]
        D['symbol']=symbol
        D=D[['symbol','open_date_time','open','high','low','close','volume','num_trades','taker_base_vol','taker_quote_vol']]
        
        df=D.set_index("open_date_time")
        
        df['close']=df['close'].astype(float)
        
        df=df['close']
        df1=df.resample(intervalunit).agg({
            "close":"last"
                        
        })
        
        unit =width
        band1=unit*np.std(df1['close'][len(df1)-length:len(df1)])
        bb_center=np.mean(df1['close'][len(df1)-length:len(df1)])
        band_high=bb_center+band1
        band_low=bb_center-band1
    return band_high,bb_center,band_low

In [4]:
# Authenticate to Binance
api_key = ''
api_secret = ''
client = Client(api_key, api_secret)

In [5]:
info=client.get_account()
k=info['balances']
print(k[0])
#bollingerband length and width
length=20
width=2

{'asset': 'BTC', 'free': '0.00000000', 'locked': '0.00000000'}


In [6]:
listofcoins=[i['asset'] for i in k]


In [ ]:
# Main loop
while True:
    for i in range(len(listofcoins)):
        try:
            tradePair=listofcoins[i]+'USDT'
            # Find last price
            trades = client.get_my_trades(symbol=tradePair)
            trades = trades[len(trades)-1]
            lastprice = float(trades['price'])
            price = client.get_ticker(symbol=tradePair)
            price=format(float(price['askPrice']), '.6f')
            #findRSI
            klines = client.get_klines(symbol=tradePair, interval='1m', limit='500')
            klines2 = client.get_historical_klines(tradePair, Client.KLINE_INTERVAL_1MINUTE, "1 day ago UTC")
            close = [float(entry[4]) for entry in klines]
            close_array = np.asarray(close)
            close_finished = close_array[:-1]
            rsi = computeRSI (close_finished, 14)
            #find BB
            bb_1m=bollingerband(tradePair,width,'1T',length)
            print(rsi,bb_1m,tradePair)
            if ((rsi<25) and (bb_1m[2]>float(price))):
                print(rsi,tradePair,price,bb_1m[2])
                balance = client.get_asset_balance(asset = 'USDT')
                coiNumber = 50
                coiprice = price
                print(coiprice)
                test = float(coiNumber)/float(coiprice)
                test=format(float(test),'.1f')
                stat = 'buy'
                #order = client.order_limit_buy(symbol=tradePair,quantity=test,price= coiprice)
                print(stat)
                sellofcoins = listofcoins[i]
                lasprice = coiprice
        except:
            pass
            #print('an error occured & retrying now')

# Repeat the code every 1 minute
    time.sleep(60)

48 (36231.67495817113, 36148.957, 36066.23904182888) BTCUSDT
58 (331.2454450924045, 330.36899999999997, 329.49255490759543) BNBUSDT
46 (0.8392367287888849, 0.833625, 0.828013271211115) WTCUSDT
45 (0.3461115798741587, 0.3452915000000001, 0.3444714201258415) LRCUSDT
52 (0.07279296077677228, 0.07269100000000002, 0.07258903922322776) TRXUSDT
53 (0.02322607705927209, 0.023124250000000006, 0.023022422940727923) FUNUSDT
52 (0.32578851748684867, 0.3250165000000001, 0.3242444825131515) CVCUSDT
55 (23.209603436727303, 23.100849999999998, 22.992096563272693) REPUSDT
51 (0.5553112495564231, 0.5535500000000001, 0.5517887504435771) NULSUSDT
35 (0.9873705413387632, 0.9812049999999999, 0.9750394586612365) STXUSDT
52 (67.26986695946218, 66.73100000000001, 66.19213304053784) ETCUSDT
62 (0.7334646934141599, 0.72951, 0.7255553065858401) BATUSDT
48 (1.635002457994082, 1.6230250000000002, 1.6110475420059183) ENJUSDT
65 (0.9032586889475779, 0.8980249999999999, 0.8927913110524218) XRPUSDT
53 (1.05057699496221

41 (0.041786585198061596, 0.04156499999999999, 0.041343414801938386) LINAUSDT
56 (0.7144894911581614, 0.7025999999999999, 0.6907105088418384) SUPERUSDT
42 (0.3282342979517755, 0.32665, 0.3250657020482245) CFXUSDT
65 (1.5691263535810955, 1.560445, 1.5517636464189046) TKOUSDT
52 (0.8770778019323254, 0.8700499999999998, 0.8630221980676741) EPSUSDT
49 (22.12595498903902, 21.8055, 21.485045010960977) FORTHUSDT
66 (8.104671171068352e-06, 8.006000000000001e-06, 7.90732882893165e-06) SHIBUSDT
52 (36170.05818958396, 36122.83099999999, 36075.603810416025) BTCUSDT
57 (332.6707445374834, 331.2005, 329.73025546251654) BNBUSDT
46 (0.8367801988293179, 0.832065, 0.8273498011706822) WTCUSDT
51 (0.34828965761041036, 0.3460245, 0.34375934238958966) LRCUSDT
52 (0.07281699411744351, 0.072732, 0.0726470058825565) TRXUSDT
56 (0.02328649897118906, 0.023164999999999998, 0.023043501028810937) FUNUSDT
56 (0.3263098727849513, 0.32534699999999994, 0.32438412721504856) CVCUSDT
62 (23.252858997509293, 23.15629999999

52 (0.014551366535961346, 0.014512049999999999, 0.014472733464038651) CKBUSDT
53 (7.564876687167923, 7.4806, 7.3963233128320764) FIROUSDT
49 (3.207819178413569, 3.19437, 3.1809208215864313) LITUSDT
59 (1.0340435175158915, 1.029255, 1.0244664824841085) SFPUSDT
54 (1.6750542961540256, 1.6622499999999998, 1.649445703845974) DODOUSDT
42 (7.503106106158683, 7.476900000000002, 7.450693893841322) ACMUSDT
42 (0.17956091808065575, 0.17809, 0.17661908191934425) OMUSDT
40 (8.630057955215905, 8.5925, 8.554942044784093) DEGOUSDT
43 (0.041698910462845185, 0.04146999999999999, 0.0412410895371548) LINAUSDT
74 (0.7317982757979984, 0.7085999999999999, 0.6854017242020014) SUPERUSDT
40 (0.3276, 0.3268, 0.32599999999999996) CFXUSDT
51 (1.5719807267839463, 1.5639999999999998, 1.5560192732160534) TKOUSDT
75 (0.9104651706790885, 0.8788999999999998, 0.8473348293209111) EPSUSDT
53 (22.11807932160713, 21.86535, 21.61262067839287) FORTHUSDT
50 (8.123426614458456e-06, 8.04e-06, 7.956573385541543e-06) SHIBUSDT
53 (

55 (16.343214781025516, 16.2742, 16.205185218974485) PSGUSDT
62 (3.5083122755027256, 3.489589999999999, 3.470867724497272) CELOUSDT
60 (0.527865955168799, 0.5256799999999998, 0.5234940448312007) TWTUSDT
60 (0.02657825375306887, 0.026458350000000002, 0.026338446246931133) REEFUSDT
62 (0.19826144181549255, 0.197125, 0.19598855818450744) RIFUSDT
46 (0.014561021252013173, 0.014486900000000002, 0.014412778747986832) CKBUSDT
61 (7.546055472388497, 7.485749999999999, 7.4254445276115) FIROUSDT
62 (3.214486062293954, 3.201129999999999, 3.1877739377060443) LITUSDT
56 (1.0346333626243742, 1.03084, 1.0270466373756257) SFPUSDT
46 (1.6740000000000002, 1.6620000000000001, 1.6500000000000001) DODOUSDT
55 (7.506872373820786, 7.480099999999998, 7.4533276261792105) ACMUSDT
55 (0.17904363471553722, 0.17779000000000003, 0.17653636528446284) OMUSDT
45 (8.603689889502524, 8.578350000000002, 8.55301011049748) DEGOUSDT
59 (0.041779136645896005, 0.041489999999999985, 0.041200863354103966) LINAUSDT
49 (0.7278530